In [21]:
import math 
from pandas_datareader import data, wb
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import numpy as np

def nCr(n, r):
    f = math.factorial
    return (f(n)/f(r))/f(n-r)

In [22]:
def fetchData(ticker = "GME"):
    data = pdr.get_data_yahoo(ticker)
    # print(data)
    data_AdjClose = data["Adj Close"]
    data_diff = data_AdjClose.diff()
    data_ret = data_diff/data_AdjClose
    ret_pos =[]
    ret_neg =[]
    for dt in data_ret:
        if dt >0:
            ret_pos.append(dt)
        elif dt < 0:
            ret_neg.append(dt)
        else:
            pass
    u = np.average(ret_pos)
    d = np.average(ret_neg)
    data_close = data["Close"]
    s0 = data_close[-1]
    sigma = np.std(data_ret)
    return u, d, s0, sigma

In [23]:
fetchData()

(0.03961807180162902,
 -0.04030179316678688,
 135.9499969482422,
 0.08464240330666838)

In [24]:
class OptionCRR():
    """
    This function find the option price under CRR Model.
    """
    
    def __init__(self, s0, sigma, strike, maturity, rfr, n, dyield = None):
        '''
        s0: initial equity price, sigma: volatility, rfr: risk free rate, n: number
        '''
        self.s0 = s0
        self.sigma = sigma
        self.rfr = rfr
        self.maturity = maturity
        self.strike = strike
        self.n = n
        self.dyield = dyield
        
    def price(self):
        delta = float(self.maturity)/float(self.n)
        u = math.exp(self.sigma*math.sqrt(delta))
        d= 1/math.exp(self.sigma*math.sqrt(delta))
        if self.dyield == None:
            q = (math.exp(self.rfr*delta) - d) / (u - d)
        else:
            q = (math.exp((self.rfr-self.dyield)*delta) - d) / (u - d)
        prc = 0
        temp_stock = 0
        temp_payout = 0
        for x in range(0, self.n + 1):
            temp_stock = self.s0*((u)**(x))*((d)**(self.n - x))
            temp_payout = max(temp_stock - self.strike, 0)
            prc += nCr(self.n, x)*(q**(x))*((1-q)**(self.n - x))*temp_payout
        prc = prc / ((1+ self.rfr*delta )**self.n)
        #prc = prc / math.exp(self.rfr*delta)
        return prc

In [25]:
import pandas as pd
d = pd.read_csv("GME.csv")
d

,Date,Open,High,Low,Close,Adj Close,Volume
0,2002-02-13,9.625000,10.060000,9.525000,10.050000,6.766665,19054000
1,2002-02-14,10.175000,10.195000,9.925000,10.000000,6.733001,2755400
2,2002-02-15,10.000000,10.025000,9.850000,9.950000,6.699334,2097400
3,2002-02-19,9.900000,9.900000,9.375000,9.550000,6.430017,1852600
4,2002-02-20,9.600000,9.875000,9.525000,9.875000,6.648839,1723200
...,...,...,...,...,...,...,...
5080,2022-04-19,142.850006,152.979996,139.860001,152.470001,152.470001,2354200
5081,2022-04-20,152.000000,152.330002,146.649994,148.850006,148.850006,1162000
5082,2022-04-21,150.000000,153.119995,139.910004,141.559998,141.559998,1684400
5083,2022-04-22,142.550003,146.300003,135.550003,138.220001,138.220001,1566600


In [26]:
df1 = d['Adj Close']

In [27]:
log_price = np.log(df1)

In [28]:
log_ret = log_price.diff() #log_ret = log(d2/d1), log(d3,d2) ..

In [29]:
log_ret

0            NaN
1      -0.004987
2      -0.005013
3      -0.041031
4       0.033465
          ...   
5080    0.074951
5081   -0.024029
5082   -0.050215
5083   -0.023877
5084   -0.016559
Name: Adj Close, Length: 5085, dtype: float64

In [ ]:
vol = ad